### Danh sách thư viện

In [ ]:
%pip install bitcoinlib

In [1]:
from bitcoinlib.keys import HDKey
from bitcoinlib.wallets import wallet_create_or_open, WalletTransaction
from bitcoinlib.services.services import Service
import os

## Create a Multisig 2-2 Script

### Tạo ví và lưu vào file multisig_wallets_info.txt (chạy 1 lần vì mỗi lần chạy sẽ tạo ra ví khác)

In [31]:
private_key1 = HDKey( network='testnet', multisig=True)
private_key2 = HDKey( network='testnet', multisig=True)

# Generate the public keys for both participants
pubkey1 = private_key1.public_master_multisig()
pubkey2 = private_key2.public_master_multisig()


wallet1 = wallet_create_or_open('multisig_wallet1', sigs_required=2, keys=[private_key1, pubkey2], network='testnet')
wallet2 = wallet_create_or_open('multisig_wallet2', sigs_required=2, keys=[pubkey1, private_key2], network='testnet')



##### Lưu thông tin  ví vào 1 file multisig_wallets_info.txt

In [32]:
import os

# Tên file để lưu thông tin
output_file = "multisig_wallets_info.txt"

# Lấy thông tin từ wallet1
wallet1_info = {
    "name": wallet1.name,
    "address": wallet1.get_key().address,  # Lấy địa chỉ đầu tiên
    "private_key": private_key1.wif(),  # Khóa bí mật của cosigner 1
    "public_key": private_key1.public_master_multisig(),  # Khóa công khai của cosigner 1
}

# Lấy thông tin từ wallet2
wallet2_info = {
    "name": wallet2.name,
    "address": wallet2.get_key().address,  # Lấy địa chỉ đầu tiên
    "private_key": private_key2.wif(),  # Khóa bí mật của cosigner 2
    "public_key": private_key2.public_master_multisig(),  # Khóa công khai của cosigner 2
}

# Ghi thông tin xuống file
with open(output_file, "w") as file:
    file.write("Wallet 1 Information:\n")
    file.write(f"Name 1: {wallet1_info['name']}\n")
    file.write(f"Address 1: {wallet1_info['address']}\n")
    file.write(f"Private Key 1: {wallet1_info['private_key']}\n")
    file.write(f"Public Key 1: {wallet1_info['public_key']}\n\n")
    
    file.write("Wallet 2 Information:\n")
    file.write(f"Name 2: {wallet2_info['name']}\n")
    file.write(f"Address 2: {wallet2_info['address']}\n")
    file.write(f"Private Key 2: {wallet2_info['private_key']}\n")
    file.write(f"Public Key 2: {wallet2_info['public_key']}\n\n")

print(f"Thông tin của hai ví đã được lưu vào file: {output_file}")

Thông tin của hai ví đã được lưu vào file: multisig_wallets_info.txt


##### Hàm lấy thông tin ví từ file txt

In [33]:
def load_wallet_details(file_path):
    wallet_details = {}
    with open(file_path, "r") as file:
        for line in file:
            if ": " in line: 
                key, value = line.strip().split(": ", 1)
                wallet_details[key] = value
    return wallet_details

## Lock Funds

### Sử dụng Bitcoin testnet faucet để gửi BTC đến ví vừa tạo 

Truy cập : https://coinfaucet.eu/en/btc-testnet4/ và
dán địa chỉ Multisig Wallet vừa tạo để nạp tbtc vào ví

Truy cập : https://blockstream.info/testnet và sử dụng địa chỉ từ file txt để kiểm tra Transaction của ví

***Hãy đảm bảo ví đã nhận được tbtc trước khi thực hiện Spend Locked Funds***

## Spend Locked Funds

#### Tạo 1 ví mới để nhận tBTC từ Multisig Wallet

In [ ]:
receiver_key = HDKey(network= 'testnet')
receiver_name = "Receiver Wallet"
receiver_wallet = wallet_create_or_open(receiver_name, receiver_key, network="testnet")

receiver_wallet.info()


=== WALLET ===
 ID                             50
 Name                           Receiver Wallet
 Owner                          
 Scheme                         bip32
 Multisig                       False
 Witness type                   segwit
 Main network                   testnet
 Latest update                  None

= Wallet Master Key =
 ID                             190
 Private                        True
 Depth                          0

- NETWORK: testnet -
- - Keys
  195 m/84'/1'/0'/0/0              tb1qyf5z8hwprt7l9kt8h0q3k2r89fyrmxdfvnlxq4    address index 0                     0.00000000 TBTC

- - Transactions Account 0 (0)

= Balance Totals (includes unconfirmed) =


address :  tb1qyf5z8hwprt7l9kt8h0q3k2r89fyrmxdfvnlxq4


### Load thông tin ví từ file Multisig_wallets_info rồi tạo và gửi giao dịch

***Hãy đảm bảo balance của ví trước khi thực hiện giao dịch chuyển coint***

In [ ]:

file_path = 'multisig_wallets_info.txt'
wallet_details = load_wallet_details(file_path)
wallet_name_1 = wallet_details["Name 1"] 
private_key_1 = wallet_details["Private Key 1"]
public_key_1 = wallet_details["Public Key 1"]  
wallet_name_2 = wallet_details["Name 2"]  
private_key_2 = wallet_details["Private Key 2"] 
public_key_2 = wallet_details["Public Key 2"] 

key1 = HDKey(private_key_1, network="testnet",multisig=True)
key2 = HDKey(private_key_2, network="testnet",multisig=True)


wallet1 = wallet_create_or_open(wallet_name_1, sigs_required=2, keys=[key1, public_key_2], network="testnet")
print("Wallet 1 UTXOs:", wallet1.utxos().to_json())
wallet2 = wallet_create_or_open(wallet_name_2, sigs_required=2, keys=[public_key_1, key2], network="testnet")

wallet1.utxos_update()


wallet2.utxos_update()

recipient_address = receiver_wallet.get_key().address
t = wallet2.transaction_create([(recipient_address, 10000)])
t.sign()
t2 = wallet1.transaction_import(t)
t2.sign()
print("%s == %s: %s" % (t.outputs[1].address, t2.outputs[1].address, t.outputs[1].address == t2.outputs[1].address))

t2.send(broadcast=True)
print("\n--------------------------------")
t2.info()


Wallet 1 UTXOs: [{'transaction_id': 10, 'address': 'tb1qe74axxwj084wvw3jaljecvqxuhn9e6en3j0jsch00gjct0lsp0lq96mfpt', 'script_type': 'p2wsh', 'spent': False, 'spending_index_n': None, 'key_id': 210, 'output_n': 1, 'script': b'', 'value': 17400, 'spending_txid': None, 'is_change': False, 'confirmations': 2, 'txid': 'bb8a9a089642a0a84a59b148ee724e4d6dda4e669bc1884714f10d34910bd449', 'network_name': 'testnet'}, {'transaction_id': 3, 'address': 'tb1qce9m69sv3nv94kdrfv8ukmsas4ywuwwyc29h2ufrh87zztce0plsqa68cj', 'script_type': 'p2wsh', 'spent': False, 'spending_index_n': None, 'key_id': 184, 'output_n': 0, 'script': b'\x00 \xc6K\xbd\x16\x0c\x8c\xd8Z\xd9\xa3K\x0f\xcbn\x1d\x85H\xee9\xc4\xc2\x8buq#\xb9\xfc!/\x19x\x7f', 'value': 33972, 'spending_txid': None, 'is_change': True, 'confirmations': 0, 'txid': 'd5bf45b3f4829b6cb8eacf227c5367e5eb9afb1e9e441e88f00afe1a7a8a1315', 'network_name': 'testnet'}]
tb1qyg60e7rch3crtcpsvycc6fe77uzryrgdhm7qa497nta0v9yv9t5s3ygt3g == tb1qyg60e7rch3crtcpsvycc6fe77uzryr

### Kiểm tra ví nhận coin 

In [43]:
receiver_wallet.utxos_update()
receiver_wallet.info()

=== WALLET ===
 ID                             50
 Name                           Receiver Wallet
 Owner                          
 Scheme                         bip32
 Multisig                       False
 Witness type                   segwit
 Main network                   testnet
 Latest update                  None

= Wallet Master Key =
 ID                             190
 Private                        True
 Depth                          0

- NETWORK: testnet -
- - Keys
  195 m/84'/1'/0'/0/0              tb1qyf5z8hwprt7l9kt8h0q3k2r89fyrmxdfvnlxq4    address index 0                     0.00000000 TBTC

- - Transactions Account 0 (1)
37d662405717788d625cbbb0d2ff35496093d00fe4779d1828a4b5f9c1d35542  tb1qyf5z8hwprt7l9kt8h0q3k2r89fyrmxdfvnlxq4        0       0.00010000 TBTC U 

= Balance Totals (includes unconfirmed) =
testnet              (Account 0)               0.00010000 TBTC


